In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['database.sqlite', 'hashes.txt', 'Reviews.csv']


In [45]:
import matplotlib.pyplot as plt
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

In [46]:
data = pd.read_csv('../input/Reviews.csv')

In [47]:
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [48]:
data.shape

(568454, 10)

In [49]:
#to remove the datapoints which are neither positive nor negative
data = data[data['Score']!=3]

In [50]:
data.shape

(525814, 10)

In [51]:
#for score>3, it is 'positive' review and score<3 is 'negative' review
def partition(x):
    if x>3:
        return 'positive'
    else:
        return 'negative'

In [52]:
data['Score'] = data['Score'].apply(partition)

In [53]:
sorted_data = data.sort_values(by='ProductId', axis=0)

In [54]:
final = sorted_data.drop_duplicates(subset={"UserId","Time","Text","ProfileName"})
final['Id'].size/sorted_data['Id'].size

0.6925890143662968

In [55]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [56]:
final['Score_n'] = final['Score'].apply(lambda x:1 if x == 'positive' else 0)

In [57]:
final_1 = final[:50000]

In [58]:
#remove the html tags and punctuations in the data
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def cleanpunc(raw_string):
    #print(raw_string)
    newstr = [char for char in raw_string.split() if char not in string.punctuation]
    #print(newstr)
    newstr = ' '.join(newstr)
    return newstr

In [59]:
final_1['Text'] = final_1['Text'].apply(cleanhtml)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
final_1['Text'] = final_1['Text'].apply(cleanpunc)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
final_data = final_1

In [62]:
X_train, X_test, y_train, y_test = train_test_split(final_data['Text'], final_data['Score_n'],test_size = 0.25, stratify = final_data['Score_n']  )

In [63]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(37500,)
(12500,)
(37500,)
(12500,)


In [64]:
#create an object of tfidf using tfidfvectorizer bigram range
tfidf_vect = TfidfVectorizer(ngram_range=(1,2))

In [65]:
final_bigram_train = tfidf_vect.fit_transform(X_train.values)
final_bigram_test = tfidf_vect.transform(X_test.values)

In [66]:
#standardize the bigram trian and test data
#create an object of standardscalar
sc = StandardScaler(with_mean= False)

In [67]:
X_train_standardized = sc.fit_transform(final_bigram_train)
X_test_standardized = sc.transform(final_bigram_test)

In [68]:
#create a simple model using default values of knn
knn = KNeighborsClassifier()
knn.fit(X_train_standardized, y_train)
y_pred = knn.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.85112
[[    4  1861]
 [    0 10635]]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1865
           1       0.85      1.00      0.92     10635

   micro avg       0.85      0.85      0.85     12500
   macro avg       0.93      0.50      0.46     12500
weighted avg       0.87      0.85      0.78     12500



The model is a huge overfit since all the test points are classifed as class 1. The data is imbalanced

In [69]:
#run another simple model by giving distance as weight parameter to handle imbalanced data
knn = KNeighborsClassifier(weights='distance')
knn.fit(X_train_standardized, y_train)
y_pred = knn.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.85184
[[   13  1852]
 [    0 10635]]
              precision    recall  f1-score   support

           0       1.00      0.01      0.01      1865
           1       0.85      1.00      0.92     10635

   micro avg       0.85      0.85      0.85     12500
   macro avg       0.93      0.50      0.47     12500
weighted avg       0.87      0.85      0.78     12500



In [70]:
#the result is slightly better as some test points are classifed as class 0

In [71]:
# a cross validation score is run to find the best hyperparamter K using f1_macro as scoring 

In [72]:
k = [5,11,25, 49,75,99]
cv_score = []
for i in k:
    knn = KNeighborsClassifier(n_neighbors=i,weights='distance')
    sc = cross_val_score(knn, X = X_train_standardized,y = y_train,scoring = 'f1_macro',cv = 5)
    cv_score.append(sc.mean())

In [73]:
cv_score

[0.46503721437941126,
 0.4635698528572772,
 0.4632023075168947,
 0.4632023075168947,
 0.4632023075168947,
 0.4632023075168947]

In [74]:
#as per the result, the K=5 seems to be the best value of hyperparamter.

**Part 2**
* To apply the logistic regression for the data and check the results and performance

In [75]:
#logistic regression with default vaules
lrmodel = LogisticRegression()
lrmodel.fit(X_train_standardized, y_train)
y_pred = lrmodel.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.88792
[[  496  1369]
 [   32 10603]]
              precision    recall  f1-score   support

           0       0.94      0.27      0.41      1865
           1       0.89      1.00      0.94     10635

   micro avg       0.89      0.89      0.89     12500
   macro avg       0.91      0.63      0.68     12500
weighted avg       0.89      0.89      0.86     12500



In [76]:
#the model is better than KNN since there are some predictions for class 0. The precision is good but the recall 
#values are poor. to handle the data imbalance, use the weight paramter in the logistic regression

In [77]:
#logistic regression with different weights to different classes
lrmodel_2 = LogisticRegression(class_weight={1:1,0:4})
lrmodel_2.fit(X_train_standardized, y_train)
y_pred = lrmodel_2.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.89144
[[  549  1316]
 [   41 10594]]
              precision    recall  f1-score   support

           0       0.93      0.29      0.45      1865
           1       0.89      1.00      0.94     10635

   micro avg       0.89      0.89      0.89     12500
   macro avg       0.91      0.65      0.69     12500
weighted avg       0.90      0.89      0.87     12500



In [78]:
#perform the cross validation for the best hyperparamter

In [79]:
C = [0.1,1,10,50,100,500,1000]
cv_score = []
for i in C:
    lrmodel = LogisticRegression(C = i)
    sc = cross_val_score(lrmodel, X = X_train_standardized,y = y_train,scoring = 'f1_macro',cv = 5)
    cv_score.append(sc.mean())

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [80]:
cv_score

[0.5721362316871588,
 0.5834533407594694,
 0.5940026244604458,
 0.5966476790783756,
 0.5967911903087295,
 0.5970733979135738,
 0.5967370566601]

In [88]:
#logistic regression with best hyper parameter values
lrmodel = LogisticRegression(C = 1000, class_weight={1:1,0:4})
lrmodel.fit(X_train_standardized, y_train)
y_pred = lrmodel.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8912
[[  546  1319]
 [   41 10594]]
              precision    recall  f1-score   support

           0       0.93      0.29      0.45      1865
           1       0.89      1.00      0.94     10635

   micro avg       0.89      0.89      0.89     12500
   macro avg       0.91      0.64      0.69     12500
weighted avg       0.90      0.89      0.87     12500



In [96]:
features = tfidf_vect.get_feature_names()
coeffs = lrmodel.coef_[0]
feats_table = pd.DataFrame(data = list(zip(features,coeffs)), columns=('features','coeffs'))

In [98]:
feats_table = feats_table.sort_values(by = 'coeffs')

In [100]:
#top 25 features for positive review
feats_table[-25:]

,features,coeffs
634445,very,0.069656
299443,is great,0.070305
261979,have,0.071342
340818,loves,0.072366
340233,love this,0.072722
588877,these,0.074870
195311,excellent,0.075449
659195,with,0.076780
44827,are,0.077551
156632,delicious,0.081218


In [101]:
#top 25 features for negative review
feats_table[0:26]

,features,coeffs
390622,not worth,-0.092628
634685,very disappointed,-0.079098
667401,would not,-0.067947
666639,worst,-0.061686
388522,not buy,-0.058806
389946,not recommend,-0.057636
165595,disappointed,-0.057090
388256,not,-0.055574
568638,terrible,-0.054940
584074,the worst,-0.054276


In [82]:
#use SGD classifier using loss = log 

In [83]:
sgdcl = SGDClassifier(loss = 'log')
sgdcl.fit(X_train_standardized, y_train)
y_pred = sgdcl.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.82728
[[1071  794]
 [1365 9270]]
              precision    recall  f1-score   support

           0       0.44      0.57      0.50      1865
           1       0.92      0.87      0.90     10635

   micro avg       0.83      0.83      0.83     12500
   macro avg       0.68      0.72      0.70     12500
weighted avg       0.85      0.83      0.84     12500



In [84]:
# the model accuracy is reduced. But the model is not overfit and there is significant prediction for class 0

In [85]:
berNB = BernoulliNB()
berNB.fit(X_train_standardized, y_train)
y_pred = berNB.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8492
[[  102  1763]
 [  122 10513]]
              precision    recall  f1-score   support

           0       0.46      0.05      0.10      1865
           1       0.86      0.99      0.92     10635

   micro avg       0.85      0.85      0.85     12500
   macro avg       0.66      0.52      0.51     12500
weighted avg       0.80      0.85      0.80     12500



svm = SVC()
svm.fit(X_train_standardized, y_train)
y_pred = svm.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

SVM applied using SGD classifier with hinge loss. 

In [86]:
sgdcl2 = SGDClassifier(loss = 'hinge')
sgdcl2.fit(X_train_standardized,y_train)
y_pred = sgdcl2.predict(X_test_standardized)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [87]:
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.82168
[[1064  801]
 [1428 9207]]
              precision    recall  f1-score   support

           0       0.43      0.57      0.49      1865
           1       0.92      0.87      0.89     10635

   micro avg       0.82      0.82      0.82     12500
   macro avg       0.67      0.72      0.69     12500
weighted avg       0.85      0.82      0.83     12500

